In [1]:
import cfscrape
from lxml import etree
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
from random import randint
from time import sleep
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from datetime import date
from itertools import cycle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pickle

## Getting links

In [67]:
chrome_options = webdriver.ChromeOptions()
chrome_options.headless=False
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [2]:
website = 'Bauhaus'

In [3]:
website_links = ['https://www.bauhaus.info/arbeitsplatten/c/10001392?q=%3AMaterial00000967%3AHolz',
                'https://www.bauhaus.info/bodenbelaege/c/10000474?q=%3AMaterial00000967%3AHolz%3AMaterial00000967%3AHolzfaser',
                'https://www.bauhaus.info/bauholz/c/10000770',
                'https://www.bauhaus.info/terrassenbau/c/10000972']

In [35]:
browser.get(website_links[3])

In [49]:
all_links = []
for s, url in enumerate(website_links):
    browser.get(url)
    sleep(2)
    
    path = '//*[@class="page-title__variant"]'
    num_products = browser.find_elements_by_xpath(path)[0].text
    num_products = int(re.findall(r'\d+', num_products)[0])
    

    print('\nSection', s+1, "Products", num_products)
    
    path = '//*[@class="product-list-tiles-wrapper"]//li//h3//a'
    page_links_raw = browser.find_elements_by_xpath(path)
    
    while len(page_links_raw) < num_products:
        sleep(2)
        path = '//*[@id="main"]/section/div/button'
        browser.find_elements_by_xpath(path)[0].click()

        sleep(2)
        path = '//*[@class="product-list-tiles-wrapper"]//li//h3//a'
        page_links_raw = browser.find_elements_by_xpath(path)
    
    page_links=[]
    for url in page_links_raw:
        link = url.get_attribute('href')
        if link not in page_links:
            page_links.append(link)
        
    print('Section Links', len(page_links))
    
    all_links.extend(page_links)
    print('All Links', len(all_links))


Section 1 Products 32
Section Links 32
All Links 32

Section 2 Products 244
Section Links 244
All Links 276

Section 3 Products 346
Section Links 346
All Links 622

Section 4 Products 139
Section Links 139
All Links 761


In [53]:
with open('{}-RawLinks.pkl'.format(website), 'wb') as f:
    pickle.dump(all_links, f)

### Testing Options

In [227]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [220]:
%%time
#data = pd.DataFrame()
for i in range(445, len(all_links)): #
    sleep(2)
    url = all_links[i]
    browser.get(url)
    
    # Checking Options
    try:
        path = '//*[@class="product-variant-content categories"]//a'
        raw_options = browser.find_elements_by_xpath(path)#[0].text
    
    except:
        raw_options = ''
        pass
    
    
    # Defining Options
    if len(raw_options)>0:
        options_links=[]
        for opt in raw_options:
            link = opt.get_attribute('href')
            if link not in options_links and link!=None and len(link)>0 and link not in data['URL'].unique():
                options_links.append(link)
        
        print('\nLink #', i, '| Number of options', len(options_links))
        
        # Getting Options
        section_data = pd.DataFrame()
        for link in options_links:
            browser.get(link)
            sleep(2)
            page_data = get_page_data()
            page_data = cols_dedup(page_data)
            section_data = pd.concat([section_data, page_data], ignore_index=True)
        
        section_data['Option Type'] = "Buttons"
        data = pd.concat([data, section_data], ignore_index=True)
        print('Section Data Size', section_data.shape)
        
    
    else:
        print('\nLink #', i, '| One option')
        
        page_data = get_page_data()
        page_data['Option Type'] = "Plain"
        page_data = cols_dedup(page_data)
        data = pd.concat([data, page_data], ignore_index=True)
        print('Page Data Size', page_data.shape)
    
    print('All Data Size', data.shape)


Link # 445 | Number of options 9
Section Data Size (9, 23)
All Data Size (611, 91)

Link # 446 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 447 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 448 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 449 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 450 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 451 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 452 | Number of options 0
Section Data Size (0, 1)
All Data Size (611, 91)

Link # 453 | Number of options 3
Section Data Size (3, 22)
All Data Size (614, 91)

Link # 454 | Number of options 0
Section Data Size (0, 1)
All Data Size (614, 91)

Link # 455 | One option
Page Data Size (1, 21)
All Data Size (615, 91)

Link # 456 | Number of options 0
Section Data Size (0, 1)
All Data Size (615, 91)

Link # 457 |


Link # 544 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 545 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 546 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 547 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 548 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 549 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 550 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 551 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 552 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 553 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 554 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Link # 555 | Number of options 0
Section Data Size (0, 1)
All Data Size (728, 92)

Lin


Link # 645 | One option
Page Data Size (1, 22)
All Data Size (775, 107)

Link # 646 | Number of options 3
Section Data Size (3, 27)
All Data Size (778, 107)

Link # 647 | One option
Page Data Size (1, 18)
All Data Size (779, 107)

Link # 648 | Number of options 2
Section Data Size (2, 26)
All Data Size (781, 108)

Link # 649 | One option
Page Data Size (1, 20)
All Data Size (782, 108)

Link # 650 | One option
Page Data Size (1, 22)
All Data Size (783, 109)

Link # 651 | One option
Page Data Size (1, 25)
All Data Size (784, 109)

Link # 652 | Number of options 2
Section Data Size (2, 21)
All Data Size (786, 109)

Link # 653 | One option
Page Data Size (1, 17)
All Data Size (787, 109)

Link # 654 | Number of options 3
Section Data Size (3, 25)
All Data Size (790, 109)

Link # 655 | One option
Page Data Size (1, 21)
All Data Size (791, 109)

Link # 656 | Number of options 2
Section Data Size (2, 25)
All Data Size (793, 109)

Link # 657 | Number of options 2
Section Data Size (2, 24)
All 


Link # 749 | One option
Page Data Size (1, 23)
All Data Size (880, 121)

Link # 750 | One option
Page Data Size (1, 21)
All Data Size (881, 121)

Link # 751 | Number of options 0
Section Data Size (0, 1)
All Data Size (881, 121)

Link # 752 | One option
Page Data Size (1, 25)
All Data Size (882, 121)

Link # 753 | One option
Page Data Size (1, 18)
All Data Size (883, 121)

Link # 754 | One option
Page Data Size (1, 16)
All Data Size (884, 121)

Link # 755 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)

Link # 756 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)

Link # 757 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)

Link # 758 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)

Link # 759 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)

Link # 760 | Number of options 0
Section Data Size (0, 1)
All Data Size (884, 121)
CPU times: user 32.1 s, sys: 2.66 s, total: 34.

In [678]:
[x for x in all_links if x not in data['URL'].unique()]

['https://www.bauhaus.info/arbeitsplatten/cucine-kuechenarbeitsplatte-nach-mass/p/27523162']

In [224]:
data.to_csv('{}-RawData.csv'.format(website))

### Getting Data Functions

In [205]:
def get_page_data():
    # Title
    path = '//*[@itemprop="name"]'
    title = browser.find_elements_by_xpath(path)[0].text
    
    # Sub Title
    try:
        path = '//*[@class="subheadline"]'
        sub_title = browser.find_elements_by_xpath(path)[0].text
    except:
        sub_title = np.nan
        
    # Chosen Option
    try:
        path = "//*[@class='category-variant']"
        chosen_opt = browser.find_elements_by_xpath(path)[0].text
    except:
        chosen_opt = np.nan
        
    # Short Descr
    try:
        path = '//*[@class="desc"]'
        short_desc = browser.find_elements_by_xpath(path)[0].text
    except:
        short_desc = np.nan
        
    # Price
    path = '//*[contains(@class, "price-block")]'
    raw_price = browser.find_elements_by_xpath(path)[0].text
    
    # Item Price
    path = '//*[@class="priceValue"]'
    price_major_val = browser.find_elements_by_xpath(path)[0].text
    path = '//*[@class="comma-values"]'
    price_minor_val = browser.find_elements_by_xpath(path)[0].text
    item_price = price_major_val + '.' + price_minor_val
    
    # Meta price
    path = '//*[@class="additional-info-after-price"]'
    meta_price = browser.find_elements_by_xpath(path)[0].text
    
    # SKU
    path = '//*[@class="product-id"]'
    sku = browser.find_elements_by_xpath(path)[0].text
    
    # Detailed Descr
    try:
        path = '//*[contains(@class, "productDescription")]'
        long_desc = browser.find_elements_by_xpath(path)[0].text
    
    except:
        long_desc= np.nan
        pass

    # Main Data
    cols = []
    values = []
    path = '//*[@class="featureRow"]' #-key

    for el in browser.find_elements_by_xpath(path):
        col = el.text
        cols.append(col)

    path = '//*[@class="valueRow"]'
    for el in browser.find_elements_by_xpath(path):
        value = el.text
        values.append(value)


    page_data = pd.DataFrame([values], columns=cols)
    
    page_data['Title'] = title
    page_data['SubTitle'] = sub_title
    page_data['Chosen Option'] = chosen_opt
    page_data['Price Raw'] = raw_price
    page_data['Price Item'] = item_price
    page_data['Price Meta'] = meta_price
    page_data['Short Description'] = short_desc
    page_data['Long Description'] = long_desc
    page_data['SKU'] = sku
    page_data['URL'] = browser.current_url
    
    return page_data

In [143]:
#Check Dupl Columns
def cols_dedup(data):
    cols = []
    dupl_cols = []
    dupl_idx = []

    for i, col in enumerate(data.columns):
        if col not in cols:
            cols.append(col)
        else:
            dupl_cols.append(col)
            dupl_idx.append(i)


    if dupl_cols:
        new_cols = data.columns.to_list()
        count=0
        for idx in dupl_idx:
            count+=1
            new_cols[idx] = new_cols[idx]+'-'+str(count)
        data.columns = new_cols
    return data

# Data Processing

In [3]:
data = pd.read_csv('{}-RawData.csv'.format(website))

In [4]:
df_proc = data.copy().reset_index(drop=True)
df_proc.shape

(884, 122)

In [5]:
df_proc = df_proc.drop_duplicates(['SKU', 'URL', 'Price Item'])
df_proc.shape

(829, 122)

### Combine all columns to mine text form

In [6]:
df_proc['Combined'] = ''
for col in df_proc.columns:
    df_proc['Combined'] = df_proc['Combined'] + '; ' + df_proc[col].astype(str)

In [7]:
df_proc['Combined'] = df_proc['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

### Keywords

In [8]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [9]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [10]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

###  Quality

In [11]:
df_proc['Quality'] = ''
for i in df_proc.index:
    df_proc['Quality'].loc[i] = []
    
    for qual in keywords['Quality'].unique():
        qual = str(qual) 
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        
        if qual!= 'nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(df_proc['Quality'].loc[i]):
            df_proc['Quality'].loc[i].append(qual)
            
    try:
        df_proc['Quality'].loc[i] = ', '.join(df_proc['Quality'].loc[i])
    except:
        pass
df_proc['Quality'] =  df_proc['Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )

df_proc['Quality'].value_counts()

Hobelfallend    22
A-Sortierung    21
B-Sortierung     4
Altholz          1
Name: Quality, dtype: int64

### Sorting Class

In [12]:
df_proc['Sorting Class'] = ''
for i in df_proc.index:
    df_proc['Sorting Class'].loc[i] = []
    
    for sort_class in keywords['Sorting Class'].unique():
        sort_class = str(sort_class)
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(df_proc['Sorting Class'].loc[i]):
            df_proc['Sorting Class'].loc[i].append(sort_class)
       
    try:
        df_proc['Sorting Class'].loc[i] = ', '.join(df_proc['Sorting Class'].loc[i])
    except:
        pass
df_proc['Sorting Class'] =  df_proc['Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
df_proc['Sorting Class'].value_counts()

S10                   62
hohe Tragfähigkeit    15
S7                     1
Name: Sorting Class, dtype: int64

## Main parameters

### Wood Types

In [13]:
df_proc['Meta Wood'] = ''
for i in df_proc.index:
    for wood in keywords['Wood Type'].unique():
        wood = str(wood)
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0:
            df_proc['Meta Wood'].loc[i] = wood

In [14]:
df_proc['Meta Wood'] = df_proc['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [15]:
df_proc['Wood Type'] = df_proc['Meta Wood']
for col in ['Material', 'Material Träger', 'Holzart', 'Materialeignung','Materialbasis']:
    df_proc['Wood Type'] = df_proc['Wood Type'].fillna(df_proc[col])

In [16]:
df_proc['Wood Type'] = df_proc['Wood Type'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [17]:
df_proc['Wood Type'].isna().sum()

1

#### Cheking wrong typs and woods

In [18]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
              'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl','Metall, Verzinkt','Kunststoff, Metall',
             'Verbundwerkstoff','Beton','Edelstahl','Normalbeton','Chrom-Vanadium-Stahl','Holz, Stahl',
             'Stahl verzinkt','Gummigranulat','Hartkunststoff','Kunststoff, Stahl','Aluminium, Holz',
             'Polyethylen','Holz, Kunststoff','Polypropylen','Moltopren-Schaumstoff','Wasserbasiert',
             'Holzfaser','PVC-frei']

In [19]:
# Percentage of wrong wood types
df_proc[(df_proc['Wood Type'].isin(wrong_wood))].shape[0]/df_proc.shape[0]*100

9.7708082026538

In [20]:
df_proc[(df_proc['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

WPC                      39
Kunststoff                9
Gummigranulat             6
Hartkunststoff            6
Aluminium                 5
Edelstahl                 4
Kunststoff, Stahl         2
Holz, Kunststoff          1
Polypropylen              1
Polyethylen               1
PVC                       1
Moltopren-Schaumstoff     1
Aluminium, Holz           1
Stahl                     1
PVC-frei                  1
Holzfaser                 1
Wasserbasiert             1
Name: Wood Type, dtype: int64

In [21]:
df_proc.shape

(829, 127)

In [22]:
df_proc = df_proc[(~df_proc['Wood Type'].isin(wrong_wood))]
df_proc.shape

(748, 127)

In [23]:
df_proc['Wood Type'].value_counts()

Fichte/Tanne               219
Holz                       171
Eiche                      132
Fichte                      86
Kiefer                      47
Douglasie                   46
Fichte/Kiefer               19
Buche                       14
Esche                        6
Bambus, Holzoberflächen      3
Lärche                       2
Bambus                       1
Cumaru                       1
Name: Wood Type, dtype: int64

### Surface Treatment

In [24]:
strong_surfaces = ['geschliffen', 'gehobelt', 'glatte Oberfläche', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 'deckend',
                   'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig', 'gespitzt', 'N+F Stirnseite',
                   'abgerundet', 'Nut und Feder']

weak_surfaces = ['roh', 'keine']

In [25]:
df_proc['Combined Surface Treat'] = ''
for i in df_proc.index:
    df_proc['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in df_proc['Combined'].loc[i].lower() and surface.lower() not in ', '.join(df_proc['Combined Surface Treat'].loc[i]):
            df_proc['Combined Surface Treat'].loc[i].append(surface)
    try:
        df_proc['Combined Surface Treat'].loc[i] = ', '.join(df_proc['Combined Surface Treat'].loc[i])
    except:
        pass

In [26]:
df_proc['Combined Surface Treat'] = df_proc['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [27]:
df_proc['Surface Treatment'] = df_proc['Holzbearbeitung']
for col in ['Kantenausführung', 'Oberflächenbehandlung','Oberflächeneigenschaft']:
    df_proc['Surface Treatment'] = df_proc['Surface Treatment'] + ', ' + df_proc[col]

In [28]:
df_proc['Surface Treatment'] = df_proc['Surface Treatment'].fillna(df_proc['Combined Surface Treat'])
df_proc['Surface Treatment'] = df_proc['Surface Treatment'].str.lower()

In [29]:
df_proc['Surface Treatment'].value_counts()

gehobelt, gefast, 4-seitig gehobelt, unbehandelt                                      121
gehobelt, gefast, unbehandelt                                                         118
sägerau, unbehandelt, scharfkantig                                                     67
wachsen                                                                                35
strukturiert                                                                           22
unbehandelt                                                                            21
sägerau, imprägniert, scharfkantig                                                     20
geschliffen, nut und feder                                                             18
gebürstet                                                                              17
geschliffen, glatte, nut und feder                                                     15
gehobelt, 2-seitig gehobelt, unbehandelt, scharfkantig                                 13
gebürstet,

### Surface Quality

In [30]:
df_proc['Surface Quality'] = ''
for i in df_proc.index:
    df_proc['Surface Quality'].loc[i] = []
    
    for surf_qual in keywords['Surface Quality'].dropna().unique():
        pat = r' {} |^{} | {}$|[a-z] {}\;'.format(surf_qual.lower(), surf_qual.lower(), surf_qual.lower(), surf_qual.lower())
        if len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and surf_qual.lower() not in ', '.join(df_proc['Surface Quality'].loc[i]):
            df_proc['Surface Quality'].loc[i].append(surf_qual)
            
    try:
        df_proc['Surface Quality'].loc[i] = ', '.join(df_proc['Surface Quality'].loc[i])
    except:
        pass
df_proc['Surface Quality'] =  df_proc['Surface Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x)

In [31]:
df_proc['Surface Quality'].value_counts()

NSI    93
SI     21
Name: Surface Quality, dtype: int64

### Drying method

In [32]:
df_proc['Combined Drying Method'] = ''
for i in df_proc.index:
    df_proc['Combined Drying Method'].loc[i] = []
    for method in keywords['Drying Method'].dropna().unique():
        method = str(method) 
        
        pat = r' {} |^{} | {}$| {}\;'.format(method.lower(), method.lower(), method.lower(), method.lower())
        if method!='nan' and len(re.findall(pat, df_proc['Combined'].loc[i].lower()))>0 and method.lower() not in ', '.join(df_proc['Combined Drying Method'].loc[i]):
            df_proc['Combined Drying Method'].loc[i].append(method)

    try:
        df_proc['Combined Drying Method'].loc[i] = ', '.join(df_proc['Combined Drying Method'].loc[i])
    except:
        pass

In [33]:
df_proc['Combined Drying Method'] =  df_proc['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [34]:
df_proc['Drying Method'] = df_proc['Restfeuchte'].fillna(df_proc['Combined Drying Method'])

In [35]:
df_proc['Drying Method'].value_counts()

14 - 18 %       115
15 % +/- 3 %    114
18 % +/- 2 %     50
14 % ± 2 %       37
16 - 18 %        35
14 % - 18 %      10
frisch            2
Name: Drying Method, dtype: int64

## Dimentions

In [36]:
df_proc[['Länge','Stärke','Breite']].isna().sum()/df_proc.shape[0]*100

Länge      1.336898
Stärke    18.716578
Breite     1.336898
dtype: float64

In [37]:
df_proc['SubTitle'].str.split(' x ', expand=True).isna().sum()/df_proc.shape[0]

0    0.000000
1    0.044118
2    0.088235
3    0.998663
4    0.998663
dtype: float64

In [38]:
title_split = df_proc['SubTitle'].str.replace(r'\, Stärke\: ',' x ').str.replace(r'^.+?(?=\d)','').str.split(' x ', expand=True)
pat = r'((?<=\d| )cm)|((?<=\d| )mm)|((?<=\d| )m(?!=m))'
mult_df = pd.DataFrame([])

for col in range(3):
    temp_split = title_split[col].str.extract(pat, expand=True)
    for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
    
    mult_df = pd.concat([mult_df, temp_split[[0]]], axis=1)

In [39]:
mult_df.columns = ['Firts Mult', 'Second Mult', 'Third Mult']
mult_df['Firts Mult'] = mult_df['Firts Mult'].fillna(mult_df['Third Mult']).fillna(mult_df['Second Mult'])
mult_df['Second Mult'] = mult_df['Second Mult'].fillna(mult_df['Third Mult'])

In [40]:
mult_df = pd.concat([mult_df, df_proc[['SubTitle','Länge','Stärke','Breite']]], axis=1)

In [41]:
mult_df['SubTitle Num'] = mult_df['SubTitle'].str.replace(r'^\D+?(?=\d)','')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=^\d)\.(?=\d\d)','')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=\d)\,(?=\d)','.')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'(?<=\d)[^0-9\.]+(?=\d)','x')
mult_df['SubTitle Num'] = mult_df['SubTitle Num'].str.replace(r'[^0-9\.x]','')

In [42]:
mult_df['Firts Mult Num'] = mult_df['Firts Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)
mult_df['Second Mult Num'] = mult_df['Second Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)
mult_df['Third Mult Num'] = mult_df['Third Mult'].fillna('-999').str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)

In [43]:
pat = r'((?<=\d| )cm)|((?<=\d| )mm)|((?<=\d| )m(?!=m))'
temp_split = mult_df['Länge'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Länge Mult'] = temp_split[0]

In [44]:
temp_split = mult_df['Stärke'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Stärke Mult'] = temp_split[0]

In [45]:
temp_split = mult_df['Breite'].str.extract(pat, expand=True)
for col in temp_split.columns[1:]:
        temp_split[0] = temp_split[0].fillna(temp_split[col])
mult_df['Breite Mult'] = temp_split[0]

In [46]:
for col in ['Länge Mult', 'Breite Mult', 'Stärke Mult']:
    mult_df[col] = mult_df[col].fillna('-99999').astype(str).str.replace('mm','1').str.replace('cm','10').str.replace('m','1000').astype(float)


In [47]:
for col in ['Länge','Stärke','Breite']:
    mult_df[col + ' Num'] = mult_df[col].str.replace(r'^\D+?(?=\d)','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=^\d)\.(?=\d\d)','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=\d)\,(?=\d)','.')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'(?<=\d)[^0-9\.]+(?=\d)','x')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].str.replace(r'[^0-9\.x]','')
    mult_df[col + ' Num'] = mult_df[col + ' Num'].astype(float)

In [48]:
mult_cols = ['Länge Mult', 'Breite Mult', 'Stärke Mult']
for i, col in enumerate(['Länge Num', 'Breite Num', 'Stärke Num']):
    mult_df[col] = mult_df[col]*mult_df[mult_cols[i]]

In [49]:
mult_df['Title Dims Num'] = np.nan
for idx in mult_df.index:
    
    mults = mult_df.loc[idx, 'Firts Mult Num':'Third Mult Num'].values
    
    raw_dims = []
    for i, dim in enumerate(mult_df['SubTitle Num'].loc[idx].split('x')):
        try:
            dim = float(dim)*mults[i]
            if dim>0:
                raw_dims.append(dim)
        except:
            pass
    mult_df['Title Dims Num'].loc[idx] = 'x'.join([str(x) for x in raw_dims])

In [50]:
import math
for idx in mult_df.index:
    title_dims = mult_df['Title Dims Num'].loc[idx].split('x')
    starke = mult_df['Stärke Num'].loc[idx]
    lange = mult_df['Länge Num'].loc[idx]
    breite = mult_df['Breite Num'].loc[idx]
    
    if math.isnan(starke) and str(lange) in title_dims and str(breite) in title_dims:
        try:
            mult_df['Stärke Num'].loc[idx] = [x for x in title_dims if x!=str(lange) and x !=str(breite)][0]
        except:
            pass

In [51]:
mult_df[['Länge Num', 'Breite Num', 'Stärke Num']].isna().sum()/mult_df.shape[0]*100

Länge Num     1.336898
Breite Num    1.336898
Stärke Num    2.540107
dtype: float64

In [52]:
df_proc = pd.concat([df_proc, mult_df[['Länge Num', 'Stärke Num','Breite Num']]], axis=1)

In [53]:
df_proc = df_proc.rename(columns = {'Länge Num':'Lange mm', 'Stärke Num':'Stärke mm', 'Breite Num':'Breite mm'})

### Price

In [54]:
df_proc['Price Piece'] = np.nan
for idx in df_proc.index:
    if 'pro m' in df_proc['Price Meta'].loc[idx] and 'pro m²' not in df_proc['Price Meta'].loc[idx]:
        df_proc['Price Piece'].loc[idx] = 'pro m'
df_proc['Price Piece'].value_counts()

pro m    108
Name: Price Piece, dtype: int64

In [55]:
df_proc['Price Item'] = df_proc['Price Item'].str.replace('\.-$','')
df_proc['Price Item'] = df_proc['Price Item'].str.replace('(?<=^\d)\.(?=\d\d\d)','')
df_proc['Price Item'] = df_proc['Price Item'].astype(float)

In [56]:
df_proc['Price Raw Proc'] = df_proc['Price Raw'].str.replace(r'\n,\n','.')

In [57]:
df_proc['Price Item Proc'] = df_proc['Price Item'].fillna(df_proc['Price Raw Proc'].str.extract(r'(^\d+\.\d+)')[0])
df_proc['Price Item Proc'] = df_proc['Price Item Proc'].astype(float)

In [58]:
df_proc[['Price Item','Price Item Proc','Price Piece', 'Price Raw Proc']].sample(3)

,Price Item,Price Item Proc,Price Piece,Price Raw Proc
706,8.95,8.95,pro m,"8.95\nMENGENVORTEIL\n8,55 €/m ab 18 m\n8,31 €/..."
388,12.60,12.60,NaN,"12.60\nMENGENVORTEIL\n11,97 €/Stück ab 22 Stüc..."
739,4.00,4.00,NaN,"4.-\nMENGENVORTEIL\n3,88 €/Stück ab 392 Stück\..."


In [59]:
df_proc.head(1)

,Unnamed: 0,Anwendungsbereich,Beständigkeit,Dekor,Material,Material Träger,Max. Zuschnittsmaß,Stärke,Weitere Eigenschaften,Breite,Länge,Title,SubTitle,Chosen Option,Price Raw,Price Item,Price Meta,Short Description,Long Description,SKU,URL,Farbe,Option Type,Einsatzbereich,Feuchtigkeitsbremse,Geeignet für,Gehschallreduzierung,Inhalt,Trittschalldämmung,Allergiker geeignet,Umwelt/Gesundheit,DIN geprüft,Akustische Eigenschaften,Design,Farbgebung,Fuge,Holzart,Norm,Nutzschicht,Oberflächeneigenschaft,Sortierung,Verbindung,Verlegeart,Zertifizierungen,Geeignet für Fußbodenheizung,Brandverhalten,Real Wood,Struktur,Mittellage/Träger,FSC geprüft,Grundfarbe,Montageservice,Rutschsicherheitsgruppe,Stuhlrollen geeignet für,Emissionsklasse,Herstellergarantiebestimmungen,Herstellergarantieumfang,Wärmedurchlasswiderstand,Qualität,Zugfeste Verbindung,Blauer Engel,Höhe,Oberflächeneigenschaft Holz,Antistatisch,Form,Montageart,Emissionsklasse Holzwerkstoffe,Rutschfestigkeit,Anzahl Lagen,Montageeigenschaft,Oberfläche,Oberflächenbehandlung,Länge Diele,Formbeständig,Typ,Ausstattung,Montage,Holzbearbeitung,Kantenausführung,Restfeuchte,Verwendung,Produktklasse,Verleimung,Euroklasse,Antibakteriell,EPD,Baustoffklasse,Holzbehandlung,Wetterschutz,Bauaufsichtliche Zulassung,Hinweise,Hitzebeständigkeit,Ausführung,Profil,Anzahl Latten,Befestigung,Größe,Inhalt ausreichend für ca.,Materialspezifizierung,Maße,Belastbarkeit,Materialzusammensetzung,Grammatur,Art Verpackung,Verbrauch,Dauerhaftigkeitsklasse,Kantenverarbeitung,Passend für,BAUHAUS Garantie,Materialbasis,Temperaturbeständigkeit,Durchmesser,Pro Planet,Kopfform,Materialeignung,Nutzungshinweis,Staubtrocken nach ca.,Überarbeitbar nach ca.,Ergänzende Gefahrenmerkmale,Gefahrenhinweise,Höhenverstellbar,Niveauausgleich,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Surface Quality,Combined Drying Method,Drying Method,Lange mm,Stärke mm,Breite mm,Price Piece,Price Raw Proc,Price Item Proc
0,0,Innen,"Hitzebeständig, Lichtecht, Reinigungsmittelbes...",Anthracite,Holz,Span,365 x 90 cm,"3,8 cm","Kratzfest, Lebensmittelunbedenklich, Pflegelei...",90 cm,365 cm,CUCINE Küchenarbeitsplatte nach Maß,"Anthracite, Max. Zuschnittsmaß: 365 x 90 cm, S...",Dekor: Anthracite,"57\n,\n51\npro m (1 m = 57,51 €)\ninkl. MwSt.",57.51,"pro m (1 m = 57,51 €)\ninkl. MwSt.","Kratz-, stoß- und abriebfest\nLichtecht\nReini...",Die CUCINE Küchenarbeitsplatte in modernen Des...,Prod.Nr. 27524688,https://www.bauhaus.info/arbeitsplatten/cucine...,NaN,Buttons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0; Innen; Hitzebeständig, Lichtecht, Reinigung...",NaN,NaN,NaN,Holz,NaN,NaN,NaN,NaN,NaN,3650.0,38,900.0,pro m,"57.51\npro m (1 m = 57,51 €)\ninkl. MwSt.",57.51


In [60]:
df_proc['Price Final'] = df_proc['Price Item Proc']
for idx in df_proc[df_proc['Price Piece']=='pro m'].index:
    item_price = df_proc['Price Final'].loc[idx]
    length = df_proc['Lange mm'].loc[idx]/1000
    df_proc['Price Final'].loc[idx]=item_price*length

In [61]:
df_proc[['Price Final','Price Raw Proc','Lange mm','Price Piece']].sample(3)

,Price Final,Price Raw Proc,Lange mm,Price Piece
444,1.50,"1.50\nMENGENVORTEIL\n1,42 €/Stück ab 220 Stück...",2500.0,NaN
134,206.72,"206.72\nMENGENVORTEIL\n200,54 €/Karton ab 32 K...",2200.0,NaN
154,104.68,"104.68\nMENGENVORTEIL\n102,07 €/Karton ab 36 K...",2400.0,NaN


### SKU

In [62]:
df_proc['SKU'] = df_proc['SKU'].str.replace(r'[^\d]','')

### Images

In [ ]:
browser =  webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver', chrome_options=chrome_options)

In [112]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':np.nan}, index=[df_proc.index])
images_df.shape

(748, 2)

In [118]:
start=0
#end=60
for p, idx in enumerate(images_df.index[start:], start=start):
    
    print('\nProduct', p)
    
    
    url = df_proc['URL'].loc[idx]
    browser.get(url)
    sleep(1)
    
    img_urls = []
    path = '//*[contains(@class, "prodThumbs")]//li'
    img_num = len(browser.find_elements_by_xpath(path))

    for i in range(1, img_num+1):

        path = '/html/body/div[4]/div[4]/div[2]/div[2]/div[2]/article/section[2]/div[2]/div[1]/div[2]/ul/li[{}]/img'.format(i)
        try:
            img_url = browser.find_elements_by_xpath(path)[0].get_attribute('src')
            print(img_url)
            img_urls.append(img_url)
        except:
            pass
    
    if p%100==0:
        images_df.to_csv('Images-{}'.format(website))
    
    images_df['URL'].loc[idx]=url
    images_df['Images'].loc[idx]= '; '.join(img_urls)


Product 717
https://media.bahag.cloud/m/478047/12.jpg
https://media.bahag.com/assets/resp_product/19/03/90190393_21198760.jpg

Product 718
https://media.bahag.cloud/m/150196/12.jpg
https://media.bahag.com/assets/resp_product/60/22/90602245_21549666.jpg

Product 719
https://media.bahag.com/assets/resp_product/12/01/1201673_22233193.jpg
https://media.bahag.com/assets/resp_product/12/01/1201701_22233195.jpg

Product 720
https://media.bahag.com/assets/resp_product/12/01/1201673_22233193.jpg
https://media.bahag.com/assets/resp_product/12/01/1201701_22233195.jpg

Product 721
https://media.bahag.com/assets/resp_product/12/01/1201673_22233193.jpg
https://media.bahag.com/assets/resp_product/12/01/1201701_22233195.jpg

Product 722
https://media.bahag.com/assets/resp_product/42/08/4208838_26820056.jpg
https://media.bahag.com/assets/resp_product/42/08/4208847_26820057.jpg

Product 723
https://media.bahag.com/assets/resp_product/42/08/4208838_26820056.jpg
https://media.bahag.com/assets/resp_produc

### Updating Product types

In [65]:
df_proc['Product Type Comb'] = np.nan
df_proc['Product Type Comb'].loc[0] = ''

In [66]:
for i in df_proc.index:
    df_proc['Product Type Comb'].loc[i] = []
    
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in df_proc['Combined'].loc[i].lower():
            if any([key_word.lower() in x.lower() for x in df_proc['Product Type Comb'].loc[i]]):
                continue
            
            else:
                df_proc['Product Type Comb'].loc[i].append(key_word)
    df_proc['Product Type Comb'].loc[i] = ', '.join(df_proc['Product Type Comb'].loc[i])

In [67]:
df_proc['Product Type Comb'] = df_proc['Product Type Comb'].apply(lambda x: np.nan if x=='' else x)

In [68]:
wrong_products = ['Befestigungssysteme', 'Zubehör']
df_proc = df_proc[~df_proc['Product Type Comb'].isin(wrong_products)]
df_proc.shape

(748, 140)

In [69]:
df_proc['Product Type'] = df_proc['Product Type Comb']

In [70]:
df_proc['Product Type'] = df_proc['Product Type'].str.split(', ', expand=True)[0]

In [71]:
df_proc['Product Type'].value_counts()

Rahmen                   129
Leiste                   128
Diele                     98
Konstruktionsvollholz     93
Glattkantbrett            48
Schalbrett                36
Arbeitsplatte             31
Sperrholz                 28
Terrassendiele            27
Brett                     23
Brettschichtholz          21
Konstruktionsholz         19
Latte                     18
Balken                    16
Baudiele                   7
Tafel                      6
Holzfliese                 4
Unterkonstruktion          4
Pfosten                    3
Fertigparkett              2
Sparschalung               1
Holzkeil                   1
Massivholzdiele            1
Name: Product Type, dtype: int64

### Checking Products

In [72]:
df_proc[(df_proc['Product Type']=='') | (df_proc['Product Type'].isna())].shape

(4, 141)

In [73]:
df_proc[df_proc['Product Type'].isna()]['Title'].value_counts()

Stirnkantenschutz         1
Ecke X-Tiles              1
Wolfcraft Schlagholz      1
Abschlusskante X-Tiles    1
Name: Title, dtype: int64

### Matching keywords

In [74]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [75]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined', 'Surface Quality']

In [76]:
df_proc['Matched Keywords'] = np.nan
df_proc['Other Keywords'] = np.nan
df_proc = df_proc.fillna('-777')
keywords = keywords.fillna('-999')

In [77]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment','Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
df_proc = df_proc.apply(get_keywords, axis=1)

CPU times: user 15.5 s, sys: 91.6 ms, total: 15.6 s
Wall time: 16.1 s


In [78]:
df_proc[['Matched Keywords','Other Keywords']].isna().sum()/df_proc.shape[0]*100

Matched Keywords    0.26738
Other Keywords      0.26738
dtype: float64

### Cleaning NaNs



In [79]:
for col in df_proc.columns:
    df_proc[col] =  df_proc[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [80]:
df_proc[df_proc['Matched Keywords'].isna()].shape

(2, 143)

In [81]:
df_proc[(df_proc['Matched Keywords'].isna()) & 
          (df_proc['Wood Type']=='Holz')].shape

(1, 143)

### Pre-cleaning

In [82]:
for col in df_proc.columns:
    df_proc[col] = df_proc[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [83]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Desktop/UpWork/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '16TXP67RtrSjgYkMyv0aCkF4VnMB-Wxu9-rGlf_2FCOw'

In [84]:
sheets_clean = df_proc.copy()
sheets_clean.shape

(748, 143)

In [85]:
sheets_clean['Website Name'] = 'www.bauhaus.info'

In [86]:
images_df.drop_duplicates('URL').shape

(701, 2)

In [87]:
sheets_clean=pd.merge(sheets_clean, images_df.drop_duplicates('URL'), on='URL', how='left')

sheets_clean.shape[0], sheets_clean['Images'].isna().sum()

(748, 58)

In [88]:
sheets_clean.head(1)

,Unnamed: 0,Anwendungsbereich,Beständigkeit,Dekor,Material,Material Träger,Max. Zuschnittsmaß,Stärke,Weitere Eigenschaften,Breite,Länge,Title,SubTitle,Chosen Option,Price Raw,Price Item,Price Meta,Short Description,Long Description,SKU,URL,Farbe,Option Type,Einsatzbereich,Feuchtigkeitsbremse,Geeignet für,Gehschallreduzierung,Inhalt,Trittschalldämmung,Allergiker geeignet,Umwelt/Gesundheit,DIN geprüft,Akustische Eigenschaften,Design,Farbgebung,Fuge,Holzart,Norm,Nutzschicht,Oberflächeneigenschaft,Sortierung,Verbindung,Verlegeart,Zertifizierungen,Geeignet für Fußbodenheizung,Brandverhalten,Real Wood,Struktur,Mittellage/Träger,FSC geprüft,Grundfarbe,Montageservice,Rutschsicherheitsgruppe,Stuhlrollen geeignet für,Emissionsklasse,Herstellergarantiebestimmungen,Herstellergarantieumfang,Wärmedurchlasswiderstand,Qualität,Zugfeste Verbindung,Blauer Engel,Höhe,Oberflächeneigenschaft Holz,Antistatisch,Form,Montageart,Emissionsklasse Holzwerkstoffe,Rutschfestigkeit,Anzahl Lagen,Montageeigenschaft,Oberfläche,Oberflächenbehandlung,Länge Diele,Formbeständig,Typ,Ausstattung,Montage,Holzbearbeitung,Kantenausführung,Restfeuchte,Verwendung,Produktklasse,Verleimung,Euroklasse,Antibakteriell,EPD,Baustoffklasse,Holzbehandlung,Wetterschutz,Bauaufsichtliche Zulassung,Hinweise,Hitzebeständigkeit,Ausführung,Profil,Anzahl Latten,Befestigung,Größe,Inhalt ausreichend für ca.,Materialspezifizierung,Maße,Belastbarkeit,Materialzusammensetzung,Grammatur,Art Verpackung,Verbrauch,Dauerhaftigkeitsklasse,Kantenverarbeitung,Passend für,BAUHAUS Garantie,Materialbasis,Temperaturbeständigkeit,Durchmesser,Pro Planet,Kopfform,Materialeignung,Nutzungshinweis,Staubtrocken nach ca.,Überarbeitbar nach ca.,Ergänzende Gefahrenmerkmale,Gefahrenhinweise,Höhenverstellbar,Niveauausgleich,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Combined Surface Treat,Surface Treatment,Surface Quality,Combined Drying Method,Drying Method,Lange mm,Stärke mm,Breite mm,Price Piece,Price Raw Proc,Price Item Proc,Price Final,Product Type Comb,Product Type,Matched Keywords,Other Keywords,Website Name,Images
0,0,Innen,"Hitzebeständig, Lichtecht, Reinigungsmittelbes...",Anthracite,Holz,Span,365 x 90 cm,"3,8 cm","Kratzfest, Lebensmittelunbedenklich, Pflegelei...",90 cm,365 cm,CUCINE Küchenarbeitsplatte nach Maß,"Anthracite, Max. Zuschnittsmaß: 365 x 90 cm, S...",Dekor: Anthracite,"57\n,\n51\npro m (1 m = 57,51 €)\ninkl. MwSt.",57.51,"pro m (1 m = 57,51 €)\ninkl. MwSt.","Kratz-, stoß- und abriebfest\nLichtecht\nReini...",Die CUCINE Küchenarbeitsplatte in modernen Des...,27524688,https://www.bauhaus.info/arbeitsplatten/cucine...,NaN,Buttons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0; Innen; Hitzebeständig, Lichtecht, Reinigung...",NaN,NaN,NaN,Holz,NaN,NaN,NaN,NaN,NaN,3650.0,38,900.0,pro m,"57.51\npro m (1 m = 57,51 €)\ninkl. MwSt.",57.51,209.9115,"Arbeitsplatte, Leiste",Arbeitsplatte,{'Arbeitsplatte': 'Latte'},"{'Wood Type': 'Holz', 'Surface Treatment': '-7...",www.bauhaus.info,https://media.bahag.com/assets/resp_product/41...


In [89]:
ren_cols = {'Title':'Product Title',
            'Breite mm':'Width (mm)',
            'Lange mm':'Length (mm)', 
            'Stärke mm':'Thickness (mm)',
            'Price Final':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Quality', 'Surface Treatment','Drying Method', 'Surface Quality',
        'Sorting Class','SKU','Images'
       ]
sheets_clean['Product Title'] = sheets_clean['Product Title'] + ' ' + sheets_clean['SubTitle']
sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [90]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]
sheets_clean.shape

(748, 16)

In [91]:
sheets_clean = sheets_clean.drop_duplicates(['URL','SKU'])
sheets_clean.shape

(748, 16)

In [92]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

0.8021390374331552

In [93]:
d2g.upload(sheets_clean[cols].fillna('-'),
           spreadsheet_key,
           'Bauhaus',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'Bauhaus' id:0>